In [1]:
import tweepy as tp
from TwitterAPI import TwitterAPI
import pandas as pd
from textblob import TextBlob
import re
from bs4 import BeautifulSoup
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.corpora as corpora
from gensim import models
import string
import spellchecker
import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

ModuleNotFoundError: No module named 'gensim'

In [270]:
cd Desktop

[Errno 2] No such file or directory: 'Desktop'
/Users/macbookpro/Desktop


In [271]:
df2 = pd.read_csv('tweets.csv')
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN
1,2023-06-08 11:35:55+00:00,what-does-cbd-do-oil-gummies ⁦@Athens4Cannabis...,0,0,False,"Athens, Ga 4 Cannabis",Athens4Cannabis,1563271657,"Athens,Ga",2013-07-02 13:56:21+00:00,107,False,NaN,NaN,NaN,NaN,NaN
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN


In [272]:
df2['Tweets'] = df2['Tweets'].astype(str)
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN
1,2023-06-08 11:35:55+00:00,what-does-cbd-do-oil-gummies ⁦@Athens4Cannabis...,0,0,False,"Athens, Ga 4 Cannabis",Athens4Cannabis,1563271657,"Athens,Ga",2013-07-02 13:56:21+00:00,107,False,NaN,NaN,NaN,NaN,NaN
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN


In [273]:
# Filter the dataframe to exclude tweets with more than two hashtags
df2 = df2[df2['Tweets'].str.count('#') < 1]

# Display the filtered dataframe
print("Total rows are: ", df2.shape[0])
df2.head()

Total rows are:  693


,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN
5,2023-06-08 11:35:29+00:00,The New York session is influenced by economic...,0,0,False,yasir iqbal,yasir6251,1547267706101108736,NaN,2022-07-13 17:12:44+00:00,10,False,NaN,NaN,NaN,NaN,NaN


In [274]:
df2 = df2[~df2['Tweets'].str.contains('price') & ~df2['Tweets'].str.contains('whatsapp') & ~df2['Tweets'].str.contains('contact us')]
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN
6,2023-06-08 11:35:09+00:00,"Are we failing our young people, as latest fig...",0,0,False,Su Allen HR,SuSuallen1,322500495,"Bedfordshire, Hertfordshire",2011-06-23 08:28:22+00:00,280,False,NaN,NaN,NaN,NaN,NaN


In [275]:
CONTRACTION_MAP = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [276]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())
        
        if expanded_contraction is not None:
            expanded_contraction = first_char + expanded_contraction[1:]
        
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [277]:
#Pre-processing
def CleanTxt (text):
  text = str(text).lower() # Convert text to string then lower
  text = re.sub(r'\n', ' ', str(text)) # Replacing \n with whitespace
  text = re.sub(r'https?://\S+|www\.\S+', ' ', str(text)) # Remove URLs
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
  text = re.sub(r'@[\w]*', ' ', str(text)) #Remove @ mentions
  text = re.sub(r'#([^\s]+)', ' ', str(text)) # Remove hashtags
  text  = "".join([char for char in text if char not in string.punctuation])
  text = re.sub('[0-9]+', ' ', text) # Remove punctuations
  text = re.sub(r'[^a-zA-Z\s]+', ' ', str(text)) # Remove numbers
  text = expand_contractions(text)
  text = re.sub(r'\s\s+', '', str(text)) # Remove excess whitespaces
  
  # Correct misspelled words
  spell = spellchecker.SpellChecker(language='en')  # Use the English dictionary
  text = word_tokenize(text) # Split texts
  misspelled = spell.unknown(text) # Find mispelled text
  corrected_text = []
  for word in text:
      if word in misspelled:
          corrected_word = spell.correction(word)
          corrected_text.append(corrected_word)
      else:
          corrected_text.append(word)     
  corrected_text = [word for word in corrected_text if word is not None]  # Exclude None values
  text = ' '.join(corrected_text)

  stop_words = set(stopwords.words('english'))
  lemmatizer = nltk.WordNetLemmatizer()

  tokens = word_tokenize(text)

  tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
  tokens = [token for token in tokens if token not in stop_words and len(token) > 2]

  return " ".join(tokens)  

In [278]:
df2['Tokens'] = df2['Tweets'].apply(CleanTxt)

/var/folders/y3/bsw47_p10lzdnhh4zp6c_j400000gn/T/ipykernel_98862/3790679202.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags


In [279]:
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type,Tokens
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN,call faithless business leader extended improv...
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN,next election receive inaction tool keep list ...
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN,one main factor choosing employment job still ...
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN,destruction doe result benefit society suggest...
6,2023-06-08 11:35:09+00:00,"Are we failing our young people, as latest fig...",0,0,False,Su Allen HR,SuSuallen1,322500495,"Bedfordshire, Hertfordshire",2011-06-23 08:28:22+00:00,280,False,NaN,NaN,NaN,NaN,NaN,failing young people latest figure estimated e...


In [280]:
def get_sentiment_score(tweet):
    return SentimentIntensityAnalyzer().polarity_scores(tweet)

df2['VADER Polarity'] = df2['Tokens'].apply(lambda x : get_sentiment_score(x)['compound'])

def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score ==  0:
    return 'Neutral'
  else:
    return 'Positive'

# Create column for analysis
df2['VADER Sentiment'] = df2['VADER Polarity'].apply(getAnalysis)
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type,Tokens,VADER Polarity,VADER Sentiment
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN,call faithless business leader extended improv...,0.2732,Positive
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN,next election receive inaction tool keep list ...,0.6705,Positive
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN,one main factor choosing employment job still ...,-0.2023,Negative
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN,destruction doe result benefit society suggest...,0.1779,Positive
6,2023-06-08 11:35:09+00:00,"Are we failing our young people, as latest fig...",0,0,False,Su Allen HR,SuSuallen1,322500495,"Bedfordshire, Hertfordshire",2011-06-23 08:28:22+00:00,280,False,NaN,NaN,NaN,NaN,NaN,failing young people latest figure estimated e...,-0.5106,Negative


In [281]:
df2.shape[0]

686

In [282]:
df2.head(20)

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type,Tokens,VADER Polarity,VADER Sentiment
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN,call faithless business leader extended improv...,0.2732,Positive
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN,next election receive inaction tool keep list ...,0.6705,Positive
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN,one main factor choosing employment job still ...,-0.2023,Negative
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN,destruction doe result benefit society suggest...,0.1779,Positive
6,2023-06-08 11:35:09+00:00,"Are we failing our young people, as latest fig...",0,0,False,Su Allen HR,SuSuallen1,322500495,"Bedfordshire, Hertfordshire",2011-06-23 08:28:22+00:00,280,False,NaN,NaN,NaN,NaN,NaN,failing young people latest figure estimated e...,-0.5106,Negative
7,2023-06-08 11:34:52+00:00,Spent a couple days in Colorado Springs teachi...,0,0,False,Kevin Macho,KevinMacho1,1003488300324171776,"Wyoming, USA",2018-06-04 04:07:09+00:00,16,False,NaN,NaN,NaN,NaN,NaN,spent couple day colorado spring teaching empl...,0.4939,Positive
9,2023-06-08 11:34:34+00:00,@HMRCcustomers They say something to do with e...,0,0,False,Emma Williams,Emmagelli77,525675476,"Glanaman, Wales",2012-03-15 19:28:29+00:00,17,False,NaN,NaN,NaN,NaN,NaN,say something employment allowance getting sen...,-0.1027,Negative
10,2023-06-08 11:34:33+00:00,@hollyswrldx Go into the PAYE section and you’...,0,0,False,HMRC Customer Support,HMRCcustomers,2848281329,UK,2014-10-28 14:54:42+00:00,100494,False,NaN,NaN,NaN,NaN,NaN,pay section able see employment history denial,-0.1027,Negative
11,2023-06-08 11:34:27+00:00,@somnath1978 GDP growth can be unpacked as com...,0,0,False,Getting Started,Getting31063081,1420989550432690178,NaN,2021-07-30 06:08:26+00:00,87,False,NaN,NaN,NaN,NaN,NaN,gap growth unpacked combination two factor edu...,0.3818,Positive
12,2023-06-08 11:34:11+00:00,***For your information and an article for rea...,0,0,False,Aklilu Nigussie,megos_n,2154476581,Ethiopia,2013-10-25 08:46:15+00:00,282,False,NaN,NaN,NaN,NaN,NaN,information article creative talent gone emplo...,0.9100,Positive


In [283]:
df2.loc[30, 'Tweets']

'EXIT will analyse territorial inequality based on 7 interrelated axes: Social services and Health;  Education; Employment; Community and social life; Housing, environment and regeneration; Mobility and immobility; Digital Inclusiveness. More information: https://t.co/OeiAXJSgcM https://t.co/QqHgq86DOh'

In [284]:
df2.loc[30, 'Tokens']

'exit analyse territorial inequality based interrelated social service employment community social life housing environment regeneration mobility immobility digital information'

In [288]:
from gensim import corpora
import gensim
import pyLDAvis.gensim

# Tokenizing the strings
df2['Tokens'] = df2['Tokens'].apply(lambda x: x.split())  # Splitting the strings into tokens

AttributeError: 'list' object has no attribute 'split'

In [290]:
# Creating a dictionary of the tokens
dictionary = corpora.Dictionary(df2['Tokens'])

# Converting the tokens into a bag-of-words (BoW) corpus
corpus = [dictionary.doc2bow(doc) for doc in df2['Tokens']]

# Training the LDA model
lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=50, random_state=None, chunksize=100, iterations=100)

# Extracting the topics
topics = lda_model.print_topics(num_words=10)

topics


[(0,
  '0.033*"employment" + 0.017*"better" + 0.016*"benefit" + 0.014*"well" + 0.013*"economic" + 0.012*"public" + 0.012*"worker" + 0.012*"act" + 0.011*"would" + 0.011*"look"'),
 (1,
  '0.043*"employment" + 0.014*"policy" + 0.014*"still" + 0.012*"thats" + 0.012*"first" + 0.012*"doe" + 0.012*"create" + 0.010*"living" + 0.009*"today" + 0.009*"education"'),
 (2,
  '0.039*"employment" + 0.021*"economy" + 0.014*"lot" + 0.014*"coq" + 0.014*"someone" + 0.013*"final" + 0.012*"care" + 0.010*"impact" + 0.010*"report" + 0.009*"school"'),
 (3,
  '0.038*"employment" + 0.019*"right" + 0.012*"give" + 0.010*"help" + 0.010*"market" + 0.010*"housing" + 0.010*"time" + 0.009*"get" + 0.009*"would" + 0.009*"case"'),
 (4,
  '0.050*"employment" + 0.015*"opportunity" + 0.013*"could" + 0.012*"take" + 0.012*"job" + 0.011*"employer" + 0.010*"find" + 0.010*"new" + 0.010*"industry" + 0.009*"people"'),
 (5,
  '0.062*"employment" + 0.022*"people" + 0.021*"work" + 0.018*"pay" + 0.016*"full" + 0.013*"state" + 0.013*"ev

In [291]:
# Visualizing the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.201503 -0.079281       1        1  14.297465
5     -0.141634 -0.179390       2        1  13.758366
3     -0.011399 -0.077983       3        1  11.198075
8      0.134698 -0.028387       4        1  11.034860
9     -0.108620 -0.052215       5        1  10.803528
1     -0.073352  0.174843       6        1  10.034651
6      0.007301  0.051141       7        1   8.445765
0      0.019371  0.085513       8        1   7.910338
7     -0.011252  0.071475       9        1   7.601113
2     -0.016615  0.034284      10        1   4.915839, topic_info=            Term       Freq       Total Category  logprob  loglift
379         work  52.000000   52.000000  Default  30.0000  30.0000
370      company  31.000000   31.000000  Default  29.0000  29.0000
170     contract  36.000000   36.000000  Default  28.0000  28.0000
23          need  30.000000   30.000000  Default  27.0000  27.0000
88           pay  25.000000   25.000000  Default  26.0000  26.0000
...          ...        ...         ...      ...      ...      ...
2046      closed   2.129496    2.806740  Topic10  -5.4661   2.7366
2     employment  19.395471  461.814789  Topic10  -3.2570  -0.1574
53           doe   3.318801   16.751660  Topic10  -5.0224   1.3938
9         people   3.752961   66.774692  Topic10  -4.8995   0.1339
630         like   3.083042   21.751551  Topic10  -5.0961   1.0589

[448 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
34        6  0.122491    able
34        8  0.734947    able
34       10  0.122491    able
541       1  0.972443  access
1399      9  0.838240  across
...     ...       ...     ...
10        9  0.099342   young
109       1  0.440770   youth
109       2  0.088154   youth
109       3  0.264462   youth
109       4  0.176308   youth

[489 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 4, 9, 10, 2, 7, 1, 8, 3])

In [297]:
df2.head()

,Tweet date,Tweets,Likes,Retweets,Quote Status?,Username,Screenname,User ID,Location,Account creation date,Followers,Verified,Tweet country location name,Tweet country location code,Tweet city location fullname,Tweet city location name,Tweet location type,Tokens,VADER Polarity,VADER Sentiment
0,2023-06-08 11:36:02+00:00,Call from Caithness business leader for 'exten...,0,0,False,John O'Groat Journal & Caithness Courier,GroatNews,289985133,Caithness,2011-04-29 15:14:40+00:00,3943,False,NaN,NaN,NaN,NaN,NaN,"[call, faithless, business, leader, extended, ...",0.2732,Positive
2,2023-06-08 11:35:49+00:00,@Jacqui_Smith1 @IainDale @forthemanypod Our 2 ...,0,0,False,Daniel Snee,dsneeuk,1628427864616976385,England,2023-02-22 16:14:00+00:00,12,False,NaN,NaN,NaN,NaN,NaN,"[next, election, receive, inaction, tool, keep...",0.6705,Positive
3,2023-06-08 11:35:46+00:00,@1mpavidus One of my main factors for choosing...,0,0,False,Cammie 🌅🌠 PRE-ORDER WHEN THE STARS ALIGHT,inkwellofnectar,88406802,United Kingdom,2009-11-08 12:15:47+00:00,221,False,NaN,NaN,NaN,NaN,NaN,"[one, main, factor, choosing, employment, job,...",-0.2023,Negative
4,2023-06-08 11:35:46+00:00,destruction does not result in benefit to the ...,0,0,False,Old Soul🫧,nems1414,1481757647229575172,Twitter,2022-01-13 22:39:29+00:00,1390,False,NaN,NaN,NaN,NaN,NaN,"[destruction, doe, result, benefit, society, s...",0.1779,Positive
6,2023-06-08 11:35:09+00:00,"Are we failing our young people, as latest fig...",0,0,False,Su Allen HR,SuSuallen1,322500495,"Bedfordshire, Hertfordshire",2011-06-23 08:28:22+00:00,280,False,NaN,NaN,NaN,NaN,NaN,"[failing, young, people, latest, figure, estim...",-0.5106,Negative
